In [2]:
# Core data manipulation and analysis libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations and arrays

# Visualization libraries
import matplotlib.pyplot as plt  # For creating static, animated, and interactive visualizations
import seaborn as sns  # For statistical data visualization

# Machine learning libraries
from sklearn import metrics  # For model evaluation metrics
from sklearn.model_selection import train_test_split  # For splitting datasets

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import random

In [70]:
df = pd.read_csv('Cleaned_full_data.csv')
print(len(df))
df['attack_cat'] = df['attack_cat'].str.replace('Backdoors', 'Backdoor')
# Interesting here. I was getting this error when encoding but you seemed to have fixed it.
df = df[~df['sport'].astype(str).str.startswith('0x')]
df = df[~df['dsport'].astype(str).str.startswith('0x')]
print(len(df))

2540047
2539744


In [120]:
class CorrEncoder:
    """
    CorrEncoder: Takes a dataset as input and uses it for the encode function. Encodes the filtered categories then draws correlations.
    If correlation is above the threshold adds it to a new dataframe then returns the one hot encoded values with the labels.

    Initialisation:
        - data (pd.DataFrame): The Dataset that contains the target column and target label variables.
    """
    
    def __init__(self, data):
        self.data = data.copy()
        # Removes Label for the multi-class processing as it is based on the label category (threat or not).
        self.data = self.data.drop(columns=['attack_cat'])

    def encode(self, target_column, sparse_n, threshold, print_data):
        """
        encode: Takes a target column and target label to encode and draw correlations from. The target column is iterated through
        for all categories that contain more positive values than defined in sparse_n. This allows for filtering of sparse categories.
        The function then one hot encodes the given category with the static target column and draws correlations for them. If correlation
        is greater then threshold then add it to the new DataFrame. The function returns the one hot encoded categories that pass the
        threshold with the target label.

        The purpose of this function is to resolve the high cardinality problem in one hot encoding.

        Parameters:
            - target_column (string): The name of the target column. The target column should contain the various categories to encode.
            - sparse_n (integer): The minimum amount of positive values required for a category after encoding (deals with sparse categories).
            - threshold (float): The threshold for correlation. The function creates onehot encoded columns of all variables that have correlation
              higher that the threshold to the target label.

        Returns:
            - ohe_df (pd.DataFrame): The one hot encoded values from the target column.
        """

        self.data[target_column] = self.data[target_column].astype(str)
        value_counts = self.data[target_column].value_counts()
        # Check if number of 1s is above the given threshold set by sparse_n.
        categories = value_counts[value_counts > sparse_n].index.tolist()
        ohe_list = []
        # Attack category for multi-class binary.
        attack_cat = self.data['label']
        
        # Go through each unique category in the target column.
        for c in categories:
            col_name = f'{target_column}_{c}'

            # Create the binary encoding column for the current category and target label.
            corr_column = (self.data[target_column] == c).astype(int)
            correlation = corr_column.corr(attack_cat)

            # Check if absolute correlation is greater than threshold.
            if abs(correlation) > threshold:
                corr_column.name = col_name
                ohe_list.append(corr_column)
        if print_data:
            print('Number of Encoded Features for', target_column)
            print(len(ohe_list))
        if ohe_list:
            # NOTE: This section can be expanded to include print outs but at the moment am focusing on the evaluations.
            ohe_df = pd.concat(ohe_list, axis=1)
            return ohe_df
        else:
            return pd.DataFrame()

In [ ]:
# Import cleaned data set. I get a hex string error in sport and dsport on my computer not sure if everyone else does too.
df = pd.read_csv('Cleaned_full_data.csv')
print(len(df))
df['attack_cat'] = df['attack_cat'].str.replace('Backdoors', 'Backdoor')
df = df[~df['sport'].astype(str).str.startswith('0x')]
df = df[~df['dsport'].astype(str).str.startswith('0x')]
df['ct_ftp_cmd'] = df['ct_ftp_cmd'].fillna(0)
df = df.reset_index(drop=True)
print(len(df))

2540047
2539744


In [ ]:
# Sample the dataset to speed up process.
df = sample_data = df.sample(n=1000, random_state=42) 

In [ ]:
categorical_columns = ['state', 'service']
# There is not many unique values here so it works ok.
encoder = OneHotEncoder(sparse_output=False, dtype='float32')
encoded_data = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns), index=df.index)
full_encoded = pd.concat([df.drop(columns=categorical_columns), encoded_df], axis=1)

# Select correlation threshold. All values with correlation less than threshold to the label are not encoded.
# NOTE: Decreasing threshold 0.01 or lower significantly increases the number of columns. I think 0.01 was around 244 for downsampled but it usually is more for
# full data. Set True to print the number of encoded columns thhat passed the check.
threshold = 0.1
encoder = CorrEncoder(full_encoded)
ohe1 = encoder.encode('dsport', 30, threshold, True)
ohe2 = encoder.encode('proto', 30, threshold, True)
ohe3 = encoder.encode('sport', 30, threshold, True)
ohe4 = encoder.encode('srcip', 30, threshold, True)
ohe5 = encoder.encode('dstip', 30, threshold, True)
cols_to_drop = ['dsport', 'proto', 'sport', 'srcip', 'dstip']
filtered_data = full_encoded.drop(columns=cols_to_drop)
combined_data = pd.concat([filtered_data, ohe1, ohe2, ohe3, ohe4, ohe5], axis=1)

Number of Encoded Features for dsport
3
Number of Encoded Features for proto
3
Number of Encoded Features for sport
3
Number of Encoded Features for srcip
14
Number of Encoded Features for dstip
19


In [ ]:
# Verify data, here if the process fails for any reason (fixed index error), there will be null values.
# NOTE: If the index is not reset then correlation is calculated based on the internal index and will only
# compare the matching values.
print(combined_data.head())

              dur  sbytes  dbytes  sttl  dttl  sloss  dloss         sload  \
1410676  0.447346     568     312    31    29      0      0  7.618264e+03   
2393246  0.004443     568     304    31    29      0      0  7.670492e+05   
1061153  0.048964    4014   59366    31    29      7     28  6.461890e+05   
1062617  0.305216    4888    2872    31    29      7      7  1.231914e+05   
2301240  0.000007     264       0    60     0      0      0  1.508571e+08   

                dload  spkts  ...  dstip_149.171.126.8  dstip_149.171.126.18  \
1410676  4.184680e+03      4  ...                    0                     0   
2393246  4.105334e+05      4  ...                    0                     0   
1061153  9.560983e+06     68  ...                    0                     0   
1062617  7.239462e+04     26  ...                    0                     0   
2301240  0.000000e+00      2  ...                    0                     0   

         dstip_149.171.126.15  dstip_149.171.126.14  dst

# Example usage of Downsampling.
- When looking at the difference between masked values and Normal values far from threats there did not seem to be significant differences. ltime and stime had some small differences. Many of the distributions are skewed so it is hard to analyse with just the mean. The mask can however be useful in the evaluation stage when using shap to group the data.

In [ ]:
# Choose a random seed.
rs = 42

# Select proportion to downsample by.
downsample = 0.5
mask = (df['label'].shift(-1) != 1) & (df['label'].shift(1) != 1)
normal_rows = df[(df['attack_cat'] == 'Normal') & mask]
percentage_to_remove = int(len(normal_rows) * downsample)
rows_to_remove = normal_rows.sample(n=percentage_to_remove, random_state=rs)
df = df.drop(rows_to_remove.index)
print(f"Downsampled Rows: {len(rows_to_remove)}")

Downsampled Rows: 1078488
